# DC Real Estate Predictor

## Predict the recommended list price for a building given the number of rooms, bedrooms, bathrooms, and building type.

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# Import DC Residentail ONLY for now
from pandas.io.json import json_normalize

dc_res_url = 'https://opendata.arcgis.com/datasets/c5fb3fbe4c694a59a6eef7bf5f8bc49a_25.geojson'
    
json = requests.get(dc_res_url).json()

results = json['features']


In [3]:
# Karl Lorey - create temporary dataframe and keep only required columns/ rename them to keep clean.
class DataFrameFromDict(object):
    """
    Temporarily imports data frame columns and deletes them afterwards.
    """

    def __init__(self, data):
        self.df = pd.json_normalize(data)
        self.columns = list(self.df.columns.values)

    def __enter__(self):
        return self.df

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.df.drop([c for c in self.columns], axis=1, inplace=True)
        
        
with DataFrameFromDict(results) as residential:
    
    residential['bathrooms'] = residential['properties.BATHRM']
    residential['heat_d'] = residential['properties.HEAT_D']
    residential['heat_code'] = residential['properties.HEAT']
    residential['ac'] = residential['properties.AC']
    residential['total_rooms'] = residential['properties.ROOMS']
    residential['bedrooms'] = residential['properties.BEDRM']
    residential['most_recent_build'] = residential['properties.EYB']
    residential['remodel_yr'] = residential['properties.YR_RMDL']
    residential['stories'] = residential['properties.STORIES']
    residential['sale_date'] = residential['properties.SALEDATE']
    residential['price'] = residential['properties.PRICE']
    residential['structure_d'] = residential['properties.STRUCT_D']
    residential['structure'] = residential['properties.STRUCT']
    residential['kitchens'] = residential['properties.KITCHENS']
    residential['fireplaces'] = residential['properties.FIREPLACES']
    residential['land_area'] = residential['properties.LANDAREA']

residential.head()

,bathrooms,heat_d,heat_code,ac,total_rooms,bedrooms,most_recent_build,remodel_yr,stories,sale_date,price,structure_d,structure,kitchens,fireplaces,land_area
0,1.0,Hot Water Rad,13.0,N,6.0,3.0,1959,1995.0,2.0,2007/04/12 00:00:00+00,0.0,Row Inside,7.0,1.0,0.0,1560
1,2.0,Hot Water Rad,13.0,Y,6.0,3.0,1969,2011.0,2.0,2012/05/04 00:00:00+00,633000.0,Row Inside,7.0,1.0,0.0,1560
2,3.0,Forced Air,1.0,Y,6.0,3.0,1969,2007.0,2.0,2017/11/17 00:00:00+00,819000.0,Row Inside,7.0,1.0,0.0,1560
3,4.0,Forced Air,1.0,Y,15.0,5.0,1971,2013.0,2.0,2014/02/05 00:00:00+00,1200000.0,Single,1.0,1.0,1.0,2720
4,2.0,Forced Air,1.0,N,8.0,2.0,1960,NaN,2.0,1994/09/22 00:00:00+00,55000.0,Multi,2.0,2.0,0.0,1520


In [4]:
# import plotly.express as px

# fig = px.histogram(residential.price)

# fig.show()

In [5]:
# Checking null rows of data

print(residential.isnull().sum())

bathrooms              999
heat_d                 999
heat_code              999
ac                     999
total_rooms           1056
bedrooms              1010
most_recent_build        0
remodel_yr           56184
stories               1048
sale_date                0
price                16766
structure_d            999
structure              999
kitchens              1000
fireplaces            1002
land_area                0
dtype: int64


In [6]:
# Checking how the data is missing across rows

residential[residential['ac'].isna()]

,bathrooms,heat_d,heat_code,ac,total_rooms,bedrooms,most_recent_build,remodel_yr,stories,sale_date,price,structure_d,structure,kitchens,fireplaces,land_area
902,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,2019/06/07 00:00:00+00,820000.0,None,NaN,NaN,NaN,100
903,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,2019/05/09 00:00:00+00,700000.0,None,NaN,NaN,NaN,100
2461,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,1900/01/01 00:00:00+00,0.0,None,NaN,NaN,NaN,927
2462,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,1900/01/01 00:00:00+00,0.0,None,NaN,NaN,NaN,927
2463,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,1900/01/01 00:00:00+00,0.0,None,NaN,NaN,NaN,927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102495,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,2019/12/19 00:00:00+00,710000.0,None,NaN,NaN,NaN,15
102496,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,2020/02/07 00:00:00+00,647000.0,None,NaN,NaN,NaN,15
102922,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,1900/01/01 00:00:00+00,0.0,None,NaN,NaN,NaN,28
102923,NaN,None,NaN,None,NaN,NaN,0,NaN,NaN,2019/09/03 00:00:00+00,25000.0,None,NaN,NaN,NaN,28


### Since nulls are persistant across rows, dropping rows with mulitlpe nulls based on column with lowest amount of msising nulls

## Data Clean Up

### All Data

In [7]:
# Going to drop rows with nulls across dataset first since only about 1000 records are all missing same amount of data
residential = residential[residential['ac'].notnull()]
residential

,bathrooms,heat_d,heat_code,ac,total_rooms,bedrooms,most_recent_build,remodel_yr,stories,sale_date,price,structure_d,structure,kitchens,fireplaces,land_area
0,1.0,Hot Water Rad,13.0,N,6.0,3.0,1959,1995.0,2.0,2007/04/12 00:00:00+00,0.0,Row Inside,7.0,1.0,0.0,1560
1,2.0,Hot Water Rad,13.0,Y,6.0,3.0,1969,2011.0,2.0,2012/05/04 00:00:00+00,633000.0,Row Inside,7.0,1.0,0.0,1560
2,3.0,Forced Air,1.0,Y,6.0,3.0,1969,2007.0,2.0,2017/11/17 00:00:00+00,819000.0,Row Inside,7.0,1.0,0.0,1560
3,4.0,Forced Air,1.0,Y,15.0,5.0,1971,2013.0,2.0,2014/02/05 00:00:00+00,1200000.0,Single,1.0,1.0,1.0,2720
4,2.0,Forced Air,1.0,N,8.0,2.0,1960,NaN,2.0,1994/09/22 00:00:00+00,55000.0,Multi,2.0,2.0,0.0,1520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108100,2.0,Warm Cool,7.0,Y,6.0,2.0,1970,NaN,1.0,1900/01/01 00:00:00+00,NaN,Single,1.0,1.0,0.0,3150
108101,1.0,Warm Cool,7.0,Y,9.0,3.0,1975,NaN,1.0,2005/08/22 00:00:00+00,280000.0,Single,1.0,1.0,0.0,3094
108102,3.0,Forced Air,1.0,Y,9.0,4.0,1984,2017.0,1.0,2017/10/20 00:00:00+00,455000.0,Single,1.0,1.0,0.0,3234
108103,3.0,Warm Cool,7.0,Y,7.0,4.0,1970,NaN,1.0,2006/01/30 00:00:00+00,0.0,Single,1.0,1.0,0.0,3164


In [8]:
residential.isnull().sum()

bathrooms                0
heat_d                   0
heat_code                0
ac                       0
total_rooms             57
bedrooms                11
most_recent_build        0
remodel_yr           55186
stories                 49
sale_date                0
price                16763
structure_d              0
structure                0
kitchens                 1
fireplaces               3
land_area                0
dtype: int64

### Price

### Given the large amount of $0 dollar values for sales price, and that it is our target variable, I am going to drop rows with 0 housing price. This should still leave plenty of data (70K) to train ans test the mocel on, while also not using the imputed prices as target feature (estimate of an estimate)

In [9]:
# Dropping price values equal to zero, using inplace to do my best at limiting memory space

index_drops = residential[residential.price == 0].index

residential.drop(index_drops, inplace = True)

### Remodel Year

In [10]:
# Replace remodel nulls with zeros for now and convert to int

residential.remodel_yr.fillna(0, inplace = True)
residential.remodel_yr = residential.remodel_yr.round(0).astype(int)

residential.remodel_yr

1         2011
2         2007
3         2013
4            0
6         2011
          ... 
108099    2010
108100       0
108101       0
108102    2017
108104       0
Name: remodel_yr, Length: 76810, dtype: int64

## NEED to ensure that replacing date with zero is okay for model training? Put in ignore parameter for zeros in column??

### Total Rooms, Bedrooms, Stories, Price, Kitchens, Fireplaces

In [11]:
# Data correlations to see if MICE will perform well given features

residential.corr()

# Graph of correlations

# import seaborn as sns

# sns.pairplot(residential)

,bathrooms,heat_code,total_rooms,bedrooms,most_recent_build,remodel_yr,stories,price,structure,kitchens,fireplaces,land_area
bathrooms,1.000000,-0.124090,0.631724,0.635742,0.409694,0.240864,0.085813,0.431061,-0.278898,0.403346,0.376606,0.239011
heat_code,-0.124090,1.000000,0.052143,0.018780,-0.319519,-0.159812,-0.003746,-0.011831,-0.043465,0.069264,0.030708,-0.003084
total_rooms,0.631724,0.052143,1.000000,0.677684,0.164577,0.090281,0.074814,0.279877,-0.282584,0.500585,0.286157,0.244020
bedrooms,0.635742,0.018780,0.677684,1.000000,0.199410,0.133341,0.074907,0.302916,-0.252873,0.350149,0.293746,0.247491
most_recent_build,0.409694,-0.319519,0.164577,0.199410,1.000000,0.071344,0.084825,0.266634,-0.050958,-0.060309,0.203893,0.084122
remodel_yr,0.240864,-0.159812,0.090281,0.133341,0.071344,1.000000,0.021495,0.161184,0.005012,0.074581,0.154476,-0.003139
stories,0.085813,-0.003746,0.074814,0.074907,0.084825,0.021495,1.000000,0.078185,0.057957,0.038724,0.073143,-0.017468
price,0.431061,-0.011831,0.279877,0.302916,0.266634,0.161184,0.078185,1.000000,-0.129650,0.044302,0.416319,0.318067
structure,-0.278898,-0.043465,-0.282584,-0.252873,-0.050958,0.005012,0.057957,-0.129650,1.000000,-0.080394,-0.287668,-0.352700
kitchens,0.403346,0.069264,0.500585,0.350149,-0.060309,0.074581,0.038724,0.044302,-0.080394,1.000000,-0.006361,-0.019855


### When comparing price to the other features, we can see fairly weak correlations between all pairs. Hopefully clustering or regression will lead to combinations that are decent predictors of residential prices in DC. It may be worth replicating a similar methodology to the commercial properties in DC to see if they are easier or harder to predict.

### Creating table with mapping for categorical codes

In [12]:
residential.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76810 entries, 1 to 108104
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bathrooms          76810 non-null  float64
 1   heat_d             76810 non-null  object 
 2   heat_code          76810 non-null  float64
 3   ac                 76810 non-null  object 
 4   total_rooms        76771 non-null  float64
 5   bedrooms           76802 non-null  float64
 6   most_recent_build  76810 non-null  int64  
 7   remodel_yr         76810 non-null  int64  
 8   stories            76773 non-null  float64
 9   sale_date          76810 non-null  object 
 10  price              60047 non-null  float64
 11  structure_d        76810 non-null  object 
 12  structure          76810 non-null  float64
 13  kitchens           76809 non-null  float64
 14  fireplaces         76808 non-null  float64
 15  land_area          76810 non-null  int64  
dtypes: float64(9), int64(

In [15]:
# Groupby dataframe of categorical and categorical codes for mapping

# Heat codes
heat_codes = residential.groupby(['heat_d']).mean()

heat_codes = heat_codes['heat_code']


# AC codes, easy enough
residential['ac_code'] = np.where(residential.ac == 'Y', 1, 0)

# Structure codes
# Using pandas categorical dtype for structure codes

residential['structure_d'] = residential.structure_d.astype('category')

residential['structure_code'] = residential["structure_d"].cat.codes

residential

,bathrooms,heat_d,heat_code,ac,total_rooms,bedrooms,most_recent_build,remodel_yr,stories,sale_date,price,structure_d,structure,kitchens,fireplaces,land_area,ac_code,structure_code
1,2.0,Hot Water Rad,13.0,Y,6.0,3.0,1969,2011,2.0,2012/05/04 00:00:00+00,633000.0,Row Inside,7.0,1.0,0.0,1560,1,4
2,3.0,Forced Air,1.0,Y,6.0,3.0,1969,2007,2.0,2017/11/17 00:00:00+00,819000.0,Row Inside,7.0,1.0,0.0,1560,1,4
3,4.0,Forced Air,1.0,Y,15.0,5.0,1971,2013,2.0,2014/02/05 00:00:00+00,1200000.0,Single,1.0,1.0,1.0,2720,1,6
4,2.0,Forced Air,1.0,N,8.0,2.0,1960,0,2.0,1994/09/22 00:00:00+00,55000.0,Multi,2.0,2.0,0.0,1520,0,1
6,2.0,Forced Air,1.0,Y,5.0,4.0,1966,2011,2.0,2011/03/01 00:00:00+00,570000.0,Row Inside,7.0,1.0,1.0,1408,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108099,2.0,Hot Water Rad,13.0,Y,7.0,3.0,1971,2010,2.0,2010/07/28 00:00:00+00,245000.0,Semi-Detached,8.0,1.0,0.0,1600,1,5
108100,2.0,Warm Cool,7.0,Y,6.0,2.0,1970,0,1.0,1900/01/01 00:00:00+00,NaN,Single,1.0,1.0,0.0,3150,1,6
108101,1.0,Warm Cool,7.0,Y,9.0,3.0,1975,0,1.0,2005/08/22 00:00:00+00,280000.0,Single,1.0,1.0,0.0,3094,1,6
108102,3.0,Forced Air,1.0,Y,9.0,4.0,1984,2017,1.0,2017/10/20 00:00:00+00,455000.0,Single,1.0,1.0,0.0,3234,1,6


### Multiple Imputation

In [14]:
# Going to use MICE to get better estimates on price given other features

# Drop categorical

